# AmsterdamUMCdb Login & Connection Setup
This notebook provides the basic setup to connect to the AmsterdamUMCdb on Google BigQuery.
Fill in the form fields on the right and run the cells.

## 1. Project Configuration
Enter your Google Cloud Project ID and the dataset details.

In [1]:
#@title Project Configuration { display-mode: "form" }

PROJECT_ID = "datathon-484308" #@param {type:"string"}
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

import os
from google.colab import auth
from google.cloud import bigquery

# Set environment variable for Google Cloud Project
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

print(f"Project ID set to: {PROJECT_ID}")
print(f"Dataset: {DATASET_PROJECT_ID}.{DATASET_ID} (Location: {LOCATION})")

Project ID set to: datathon-484308
Dataset: amsterdamumcdb.van_gogh_2026_datathon (Location: eu)


## 2. Authentication
Run this cell to authenticate your Google account. You will need to use the account that has access to AmsterdamUMCdb.

In [2]:
#@title Authenticate User { display-mode: "form" }
auth.authenticate_user()
print('Successfully authenticated')

Successfully authenticated


## 3. Initialize BigQuery Client
This cell sets up the client and default configurations for querying.

In [3]:
#@title Initialize Client { display-mode: "form" }
from google.cloud import bigquery

# Default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=f"{DATASET_PROJECT_ID}.{DATASET_ID}")

# Create the BigQuery client
client = bigquery.Client(project=PROJECT_ID, location=LOCATION, default_query_job_config=def_config)

# Enable data table display in Colab
%load_ext google.colab.data_table
from google.colab.data_table import DataTable
DataTable.max_columns = 50
DataTable.max_rows = 20000

print("BigQuery client initialized and ready.")

BigQuery client initialized and ready.


## 5. Proof of Concept: Fluid Balance

In this section, we perform a sample fluid balance calculation for a single patient admission.

**Background:**
1.  **Data Sources:** We utilize `MEASUREMENT` entries.
    * *Input:* Intravascular (infusions), Enteral (tube feeding), and Oral.
    * *Output:* Urine, Stool, Drains (Chest, Ventricle, GI).
    * *Note:* *Insensible perspiration* (invisible loss via skin/breathing) is not documented in the database and is therefore not included here (pure Input/Output balance).
2.  **Time Logic:** A "Clinical Day" is defined here as **07:00 AM to 07:00 AM** of the following day (standard shift definition).
3.  **Patient Selection:** The code automatically searches for the first available patient admission after a selected start date to demonstrate functionality.

The result is a table showing the daily and cumulative balance for this specific hospital stay.

In [4]:
#@title Calculate Detailed Fluid Balance
import pandas as pd
from datetime import timedelta

# ---------------------------------------------------------
# 1. PARAMETERS
# ---------------------------------------------------------
SEARCH_START_DATE = '2017-01-01' #@param {type:"date"}

# Mapping with clear, short names for the table columns
# Prefixes (In/Out) help with sorting and grouping
CONCEPT_MAP = {
    # Inputs
    3037253: 'In: IV',
    3010494: 'In: Enteral',
    3006552: 'In: Oral',

    # Outputs
    3014315: 'Out: Urine',
    3011087: 'Out: Stool',
    3026556: 'Out: Chest Tube',
    3018767: 'Out: Ventricle Drain',
    21491183: 'Out: GI Drain',
    3020433: 'Out: Misc'
}

ids_string = ", ".join(map(str, CONCEPT_MAP.keys()))

# ---------------------------------------------------------
# 2. FIND VISIT
# ---------------------------------------------------------
print(f"Searching for the first admission after {SEARCH_START_DATE}...")

query_find_visit = f"""
SELECT person_id, visit_start_datetime, visit_end_datetime
FROM `visit_occurrence`
WHERE visit_start_datetime >= '{SEARCH_START_DATE}'
ORDER BY visit_start_datetime ASC
LIMIT 1
"""

try:
    df_visit = client.query(query_find_visit).to_dataframe()
except Exception as e:
    df_visit = pd.DataFrame()
    print(f"Error: {e}")

if df_visit.empty:
    print(f"❌ No visits found after {SEARCH_START_DATE}.")
else:
    person_id = df_visit.iloc[0]['person_id']
    visit_start = df_visit.iloc[0]['visit_start_datetime']
    visit_end = df_visit.iloc[0]['visit_end_datetime']
    if pd.isnull(visit_end): visit_end = pd.Timestamp.now()

    print(f"✅ Patient: {person_id} | Admitted: {visit_start}")

    # ---------------------------------------------------------
    # 3. FETCH DATA
    # ---------------------------------------------------------
    query_measurements = f"""
    SELECT measurement_datetime, measurement_concept_id, value_as_number
    FROM `measurement`
    WHERE person_id = {person_id}
      AND measurement_concept_id IN ({ids_string})
      AND measurement_datetime >= '{visit_start}'
      AND measurement_datetime <= '{visit_end}'
    ORDER BY measurement_datetime
    """

    df_data = client.query(query_measurements).to_dataframe()

    if df_data.empty:
        print("⚠️ No fluid data found for this visit.")
    else:
        # ---------------------------------------------------------
        # 4. DETAILED CALCULATION
        # ---------------------------------------------------------
        df_data['measurement_datetime'] = pd.to_datetime(df_data['measurement_datetime'])

        # Shift 07:00 AM
        df_data['clinical_date'] = (df_data['measurement_datetime'] - timedelta(hours=7)).dt.date

        # Map IDs to readable names
        df_data['component'] = df_data['measurement_concept_id'].map(CONCEPT_MAP)

        # Pivot: Rows = Date, Columns = Specific Components
        df_pivot = df_data.pivot_table(
            index='clinical_date',
            columns='component',
            values='value_as_number',
            aggfunc='sum',
            fill_value=0
        )

        # Calculate Totals based on column name prefixes
        # (This works dynamically even if some columns are missing)
        input_cols = [c for c in df_pivot.columns if c.startswith('In:')]
        output_cols = [c for c in df_pivot.columns if c.startswith('Out:')]

        df_pivot['TOTAL INPUT'] = df_pivot[input_cols].sum(axis=1)
        df_pivot['TOTAL OUTPUT'] = df_pivot[output_cols].sum(axis=1)

        # Balance
        df_pivot['BALANCE'] = df_pivot['TOTAL INPUT'] - df_pivot['TOTAL OUTPUT']
        df_pivot['CUMULATIVE'] = df_pivot['BALANCE'].cumsum()

        # Reorder columns for better readability: Inputs -> Total In -> Outputs -> Total Out -> Balance
        final_cols = input_cols + ['TOTAL INPUT'] + output_cols + ['TOTAL OUTPUT', 'BALANCE', 'CUMULATIVE']
        df_pivot = df_pivot[final_cols]

        # ---------------------------------------------------------
        # 5. CLEAN DISPLAY
        # ---------------------------------------------------------
        print(f"\n📋 Detailed Fluid Balance (ml)")

        # Simple styling: No colors, just thousands separators and 0 decimals
        display(df_pivot.style.format("{:,.0f}"))

Searching for the first admission after 2017-01-01...
✅ Patient: 52333 | Admitted: 2017-01-01 00:00:00+00:00

📋 Detailed Fluid Balance (ml)


component,In: Enteral,In: IV,In: Oral,TOTAL INPUT,Out: GI Drain,Out: Stool,Out: Urine,TOTAL OUTPUT,BALANCE,CUMULATIVE
clinical_date,,,,,,,,,,
2016-12-31,0,625,0,625,500,0,460,960,-335,-335
2017-01-01,153,"3,327",0,"3,479",20,0,"1,120","1,140","2,339","2,005"
2017-01-02,"1,292","1,562",50,"2,904",0,0,"1,280","1,280","1,624","3,629"
2017-01-03,"1,320",843,0,"2,163",0,200,"1,860","2,060",103,"3,732"
2017-01-04,"1,210","1,474",0,"2,684",0,150,122,272,"2,412","6,144"
2017-01-05,"1,320","1,466",0,"2,786",0,0,10,10,"2,776","8,920"
2017-01-06,"1,375","1,336",0,"2,711",0,0,"4,098","4,098","-1,387","7,533"
2017-01-07,275,162,0,438,0,0,538,538,-100,"7,432"
